In [1]:
# Import necessary packages
from gerrychain import (GeographicPartition, Partition, Graph, MarkovChain,
                        proposals, updaters, constraints, accept, Election, grid)
from gerrychain.metrics import mean_median, partisan_bias, polsby_popper, efficiency_gap
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import inspect
import pickle
import geopandas as gp
from shapely.geometry import LineString, Polygon
from shapely.ops import unary_union
import itertools

In [2]:
# Import tools file
from gerrymandering_tools import *

In [34]:
precincts = gp.read_file('gdf_mp_sp.shp')

In [38]:
# Get the graph data
graph_annika = Graph.from_json("../../Updated_Files/Graphs/graph_zero_2018.json")

In [39]:
graph_orig = pickle.load(open('graph_mp_sp.pkl', 'rb'))

In [40]:
len(graph_annika)-len(graph_orig)

0

In [32]:
edges_added = [ edge for edge in graph.edges if edge not in graph1.edges]
edges_removed = [ edge for edge in graph1.edges if edge not in graph.edges]

In [41]:
len(edges_removed), len(edges_added)

(7, 27)

In [52]:
list(graph_orig[2000].keys())

[1680, 1842, 1994, 2432, 2573, 2188, 2296]

In [44]:
s = unary_union([precincts.iloc[1]['geometry'], precincts.iloc[1000]['geometry']])

In [45]:
for i in range(len(precincts)):
    if type(precincts.iloc[i]['geometry']) != Polygon:
        print(i)

In [46]:
%timeit s = unary_union([precincts.iloc[1]['geometry'], precincts.iloc[1000]['geometry']])

513 µs ± 10.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [60]:
questionable_edges1 = []
for i in range(len(precincts)):
    neighbors = list(graph_orig[i].keys())
    for n in neighbors:
        if n > i:
            s = unary_union([precincts.iloc[i]['geometry'], precincts.iloc[n]['geometry']])
            if type(s) != Polygon:
                questionable_edges1.append((i, n))
    if i%50 ==0: print(i, end=', ')

0, 50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950, 1000, 1050, 1100, 1150, 1200, 1250, 1300, 1350, 1400, 1450, 1500, 1550, 1600, 1650, 1700, 1750, 1800, 1850, 1900, 1950, 2000, 2050, 2100, 2150, 2200, 2250, 2300, 2350, 2400, 2450, 2500, 2550, 2600, 

In [61]:
questionable_edges1

[(408, 409),
 (408, 452),
 (412, 439),
 (439, 484),
 (1256, 2585),
 (1946, 2190),
 (2628, 2635)]

In [58]:
edges_removed

[(408, 409),
 (408, 452),
 (412, 439),
 (439, 484),
 (1256, 2585),
 (1946, 2190),
 (2628, 2635)]

In [98]:
x, y = precincts.iloc[208]['geometry'].exterior.xy

array.array

In [79]:
questionable_edges2 = []
for i in range(len(precincts)):
    neighbors = set(graph_orig[i].keys())
    degree2 = set([i for n in neighbors for i in set(graph_orig[n].keys())])
    layer2 = degree2.difference(neighbors.union([i]))
    for n in layer2:
        if n > i:
            s = unary_union([precincts.iloc[i]['geometry'], precincts.iloc[n]['geometry']])
            if type(s) == Polygon:
                questionable_edges2.append((i, n))
    if i%50 ==0: print(i, end=', ')

0, 50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950, 1000, 1050, 1100, 1150, 1200, 1250, 1300, 1350, 1400, 1450, 1500, 1550, 1600, 1650, 1700, 1750, 1800, 1850, 1900, 1950, 2000, 2050, 2100, 2150, 2200, 2250, 2300, 2350, 2400, 2450, 2500, 2550, 2600, 

In [80]:
questionable_edges2

[]

In [73]:
i = 1
neighbors = set(graph_orig[i].keys())
degree2 = set([i for n in neighbors for i in set(graph_orig[n].keys())])

In [77]:
degree2.difference(neighbors.union([i]))

{2, 5, 7, 8, 11, 13, 14, 15, 16, 134, 135, 264, 363, 364, 615}

In [78]:
neighbors

{0, 4, 6, 9, 138}

In [68]:
graph_orig[4]

AtlasView({1: {'shared_perim': 14195.368816551376, 'id': 1}, 14: {'shared_perim': 3399.944625591862, 'id': 4}, 615: {'shared_perim': 40773.90224566298, 'id': 4}, 15: {'shared_perim': 56345.531955938, 'id': 4}, 6: {'shared_perim': 36616.26481482544, 'id': 4}, 264: {'shared_perim': 8919.337140419495, 'id': 4}, 138: {'shared_perim': 14127.721897186437, 'id': 4}})